---
title: "How to build a motion tracking pipeline: Processing MediaPipe coordinates"
authors: Gillian Rosenberg, Šárka Kadavá
---

We performed motion tracking with MediaPipe and now we have saved coordinates for each file in a csv file.

There are important things to do before we can actually start any analysis. But let's take it step by step

# Preparing an environment

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import math
import scipy
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import platform  # To detect the operating system

# Get the current working directory
curfolder = os.getcwd()

user = "Windows"

# Check the OS type
if user == "Windows":
    mtfolder = curfolder + '\\..\\..\\01_MotionCapture\\Mediapipe\\Output_TimeSeries\\'
    vidfolder = curfolder + '\\..\\..\\01_MotionCapture\\Mediapipe\\Output_Videos\\'
elif user == "Mac":  # Darwin is the system name for macOS
    mtfolder = os.path.join(curfolder, '..', '..', '01_MotionCapture', 'Mediapipe', 'Output_TimeSeries')
    vidfolder = os.path.join(curfolder, '..', '..', '01_MotionCapture', 'Mediapipe', 'Output_Videos')
else:
    raise SystemError("Unsupported operating system")

# Get all the files in the folder
mtfiles = glob.glob(os.path.join(mtfolder, '*body_world.csv'))  # Use os.path.join for compatibility
print(mtfiles)

vidfiles = glob.glob(os.path.join(vidfolder, '*.avi'))
print(vidfiles)

# Inspecting a file

Let's start by inspecting one file. (TIP: Inspect in Data Wranger)

In [ ]:
# Load in one file
sample = pd.read_csv(mtfiles[0])
sample.head(20)

What kind of columns do we see?
In what time interval are we saving the coordinates? What is our sampling rate?
What do x, y, z coordinates represent?
What is visibility?

When we familiarized ourselves with the data, we can start step by step cleaning it.

For example, let's say we don't need the visibility column. It might be useful for some specific cases, but for now, we can drop it.

In [ ]:
# Drop visibility cols
sample = sample.drop(sample.columns[sample.columns.str.contains('visibility',case = False)],axis = 1)

Now we conventiently reduced the number of columns to 100.

In [ ]:
sample.head(20)

Now let's check how do coordinates look like, for example for right wrist

In [ ]:
# Plot x, y, z for RIGHT_WRIST
sample['Y_RIGHT_WRIST'].plot()
sample['X_RIGHT_WRIST'].plot()
sample['Z_RIGHT_WRIST'].plot()
plt.legend()
plt.show()


Let's now check back to the video, so we can understand what do these coordinates represent.

In [ ]:
# this is the video we need to look at 
mtfiles[0]

What do we see? 
What does y-axis represent?
Do you notice something weird?

Yes, we need to....

-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-

# Flipping the data

In MediaPipe, for some reason the y-axis and z-axis are flipped. Who knows what is the reason, but for us this serves as a good example of why we **always** need to check our data.

In [ ]:
# method to flip the data
def flip_data(df):
    cols = df.columns
    cols  = [col for col in df.columns if 'Y_' in col]
    cols = list(cols)
        # first get the vertical height of the configuration
        # we only do this for the first frame; the transformation will be applied to all frames
    maxpoint = []
    for joint in cols:
        maxpoint.append(df.loc[0, joint])
        # iterate over each joint, in each frame, to flip the y-axis
    for frame in range(len(df)):
        for joint in cols:
            ytrans = max(maxpoint) - df.loc[frame, joint] - 1
            df.loc[frame, joint] = ytrans
    return df

# use it on the file
sample = flip_data(sample)

In [ ]:
# plot to check

sample['Y_RIGHT_WRIST'].plot()
sample['X_RIGHT_WRIST'].plot()
sample['Z_RIGHT_WRIST'].plot()
plt.legend()
plt.show()

# Interpolating the data

This will be video-specific problem, but sometimes we might encounter that there are missing data points - for example, when one arm covers the other, or when arms cover the face, etc. If these missing windows are of reasonable length, we can **interpolate** them. Interpolation is a method that estimates a missing value based on the values of the surrounding data points.

In [ ]:
# interpolate the data
def interpolate(sample):
    cols = sample.columns

    # put away time from cols
    cols = cols.drop(['time'])

    # loop over the cols and interpolate missing data
    for col in cols:
        sample[col] = sample[col].interpolate(method='linear', x = sample['time'], limit=10) # limit is the max number of missing values to fill
    
    return sample

# use it on the file
sample = interpolate(sample)


# Smoothing the data

Looking at the plots, you may notice that the data is quite noisy and jerky. This is because we are collecting each value for a keypoint at certain frame rate - for us it is ca. 16 ms. 

We can **smooth** the data with various filters - Butterworth, Savitzky-Golay, etc.

In this script, we will be using **Savitzky-Golay filter**. This filter fits a *x-order* polynomial to a sequence of data points and then evaluates the polynomial at the central point of the sequence. This is repeated for on sequence of *windows*. This allows us to smooth the data without losing the original shape of the curve.

When using this filter, we need to specify the *window* and *order* parameters. The *window* parameter specifies the number of data points used to fit the polynomial, while the *order* parameter specifies the order of the polynomial.

So now the question arises - HOW TO CHOOSE THOSE???

Once again, we are not going to blindly guess. We can just try different combinations and see which are smoothing the data approprietly, without losing too much of the original shape of the curve.

In [ ]:
# function to check different smoothing windows and orders
def check_smooth_strength(df, windows, orders, keytoplot):

    # prepare new df
    df_smooth = pd.DataFrame()

    for win in windows:
        for ord in orders:
            df_smooth[keytoplot + '_savgol' + str(win) + '_' + str(ord)] = scipy.signal.savgol_filter(df[keytoplot], win, ord)

    # make R_Hand_x from df_sample a list
    keytoplot_unsmoothed = df[keytoplot].tolist()

    # load these values into df_smooth as a new column
    df_smooth[keytoplot] = keytoplot_unsmoothed

    # plot keytoplot in all strngths
    colstoplot = [x for x in df_smooth.columns if keytoplot in x]
    plt.figure()
    for col in colstoplot:
        plt.plot(df_smooth[col], label=col)
    plt.legend()
    plt.show()

In [ ]:
windows = [15, 25, 35] # list possible window
orders = [1, 2, 3] # list possible orders

check_smooth_strength(sample, windows, orders, 'X_RIGHT_WRIST')

Can you comment on some of the different combinations?

In [ ]:
# smooth the data
def smooth_data(sample):
    cols_upperbody = ['X_NOSE', 'Y_NOSE', 'Z_NOSE', 'X_LEFT_EYE_INNER', 'Y_LEFT_EYE_INNER', 'Z_LEFT_EYE_INNER', 'X_LEFT_EYE', 'Y_LEFT_EYE', 'Z_LEFT_EYE', 'X_LEFT_EYE_OUTER', 'Y_LEFT_EYE_OUTER', 'Z_LEFT_EYE_OUTER', 'X_RIGHT_EYE_OUTER', 'Y_RIGHT_EYE_OUTER', 'Z_RIGHT_EYE_OUTER', 'X_RIGHT_EYE', 'Y_RIGHT_EYE', 'Z_RIGHT_EYE', 'X_RIGHT_EYE_OUTER.1', 'Y_RIGHT_EYE_OUTER.1', 'Z_RIGHT_EYE_OUTER.1', 'X_LEFT_EAR', 'Y_LEFT_EAR', 'Z_LEFT_EAR', 'X_RIGHT_EAR', 'Y_RIGHT_EAR', 'Z_RIGHT_EAR', 'X_MOUTH_LEFT', 'Y_MOUTH_LEFT', 'Z_MOUTH_LEFT', 'X_MOUTH_RIGHT', 'Y_MOUTH_RIGHT', 'Z_MOUTH_RIGHT', 'X_LEFT_SHOULDER', 'Y_LEFT_SHOULDER', 'Z_LEFT_SHOULDER', 'X_RIGHT_SHOULDER', 'Y_RIGHT_SHOULDER', 'Z_RIGHT_SHOULDER', 'X_LEFT_ELBOW', 'Y_LEFT_ELBOW', 'Z_LEFT_ELBOW', 'X_RIGHT_ELBOW', 'Y_RIGHT_ELBOW', 'Z_RIGHT_ELBOW', 'X_LEFT_WRIST', 'Y_LEFT_WRIST', 'Z_LEFT_WRIST', 'X_RIGHT_WRIST', 'Y_RIGHT_WRIST', 'Z_RIGHT_WRIST', 'X_LEFT_PINKY', 'Y_LEFT_PINKY', 'Z_LEFT_PINKY', 'X_RIGHT_PINKY', 'Y_RIGHT_PINKY', 'Z_RIGHT_PINKY', 'X_LEFT_INDEX', 'Y_LEFT_INDEX', 'Z_LEFT_INDEX', 'X_RIGHT_INDEX', 'Y_RIGHT_INDEX', 'Z_RIGHT_INDEX', 'X_LEFT_THUMB', 'Y_LEFT_THUMB', 'Z_LEFT_THUMB', 'X_RIGHT_THUMB', 'Y_RIGHT_THUMB']
    cols_lowerbody = ['X_LEFT_HIP',
       'Y_LEFT_HIP', 'Z_LEFT_HIP', 'X_RIGHT_HIP', 'Y_RIGHT_HIP', 'Z_RIGHT_HIP',
       'X_LEFT_KNEE', 'Y_LEFT_KNEE', 'Z_LEFT_KNEE', 'X_RIGHT_KNEE',
       'Y_RIGHT_KNEE', 'Z_RIGHT_KNEE', 'X_LEFT_ANKLE', 'Y_LEFT_ANKLE',
       'Z_LEFT_ANKLE', 'X_RIGHT_ANKLE', 'Y_RIGHT_ANKLE', 'Z_RIGHT_ANKLE',
       'X_LEFT_HEEL', 'Y_LEFT_HEEL', 'Z_LEFT_HEEL', 'X_RIGHT_HEEL',
       'Y_RIGHT_HEEL', 'Z_RIGHT_HEEL', 'X_LEFT_FOOT_INDEX',
       'Y_LEFT_FOOT_INDEX', 'Z_LEFT_FOOT_INDEX', 'X_RIGHT_FOOT_INDEX',
       'Y_RIGHT_FOOT_INDEX', 'Z_RIGHT_FOOT_INDEX']

# smooth upperbody and face with savgol 20,4
    for col in cols_upperbody:
        sample[col] = scipy.signal.savgol_filter(sample[col], 30, 1)

# smooth lowerbod with savgol 30,3
    for col in cols_lowerbody:  
        sample[col] = scipy.signal.savgol_filter(sample[col], 30, 1)
    return(sample)

# use it on the file
sample = smooth_data(sample)

# Getting derivatives

Now we are almost done! We have cleaned the data, interpolated it, smoothed it... and most likely, we want for analysis something more than just coordinates. 

Coordinates represent positional data. We can get more information from the data by calculating **the derivatives**. Derivatives represent the rate of change of a function at a given point.

First derivate of positional data represents the rate of change of the position. This means ....
Second derivate of positional data represents the rate of change of the rate of change of the position. This means ....
Third derivate of positional data represents the rate of change of the rate of change of the rate of change of the position. This means ....

Below, you can see functions that always have the same pipeline:
- collecting columns to be differentiated
- differentiating them
- smoothing the differentiated signal

Before we start, notice one thing in the code:

```python

sample[col + '_speed'] = sample[col + '_speed']*sr

```

It seems that we are multiplying the speed by the sampling rate. Why?

In [ ]:
###### 3D SPEED #############
def derive_speed_smooth(cols, sample, sr):
    speedcols = [col.replace('Y_', '') for col in cols]
    speedcols = [col.replace('X_', '') for col in speedcols]
    speedcols = [col.replace('Z_', '') for col in speedcols]

# keep only unique values
    speedcols = list(set(speedcols))

    for col in speedcols:
        x = sample['X_' + col]
        y = sample['Y_' + col]
        z = sample['Z_' + col]

    # calculate speed
        sample[col + '_speed'] = np.insert(np.sqrt(np.diff(x) ** 2 + np.diff(y) ** 2 + np.diff(z) ** 2), 0, 0)
        sample[col + '_speed'] = sample[col + '_speed']*sr
    # smooth speed:
    for col in speedcols:
        sample[col + '_speed'] = scipy.signal.savgol_filter(sample[col + '_speed'], 20, 4)

    return(speedcols, sample)

######### 2D SPEED #############
def derive_2dspeed_smooth(cols, sample, sr):
    speedcols = [col.replace('Y_', '') for col in cols]
    speedcols = [col.replace('X_', '') for col in speedcols]

    # keep only unique values
    speedcols = list(set(speedcols))

    for col in speedcols:
        x = sample['X_' + col]
        y = sample['Y_' + col]

        # calculate speed
        sample[col + '_speed2D'] = np.insert(np.sqrt(np.diff(x) ** 2 + np.diff(y) ** 2), 0, 0)
        sample[col + '_speed2D'] = sample[col + '_speed2D']*sr
        
    #  smooth speed:
    for col in speedcols:
        sample[col + '_speed2D'] = scipy.signal.savgol_filter(sample[col + '_speed2D'], 20, 4)

    return(speedcols, sample)

######### 1D VERTICAL VELOCITY #############
def derive_vertical_vel_smooth(sample):
    verticalcols = [col for col in sample.columns if 'Y_' in col]
    verticalcols = [col for col in verticalcols if 'speed' not in col]

# calculate the velocity
    for col in verticalcols:
        sample[col + '_velocity'] = np.insert(np.diff(sample[col]), 0, 0)  
    vel_cols = [col for col in sample.columns if "_velocity" in col]
    for col in vel_cols:
        sample[col] = scipy.signal.savgol_filter(sample[col], 20, 4)
    return verticalcols, sample

######### 3D ACCELERATION #############
def derive_accel_smooth(speedcols, sample):
    for col in speedcols:
        sample[col + '_acceleration'] = np.insert(np.diff(sample[col + "_speed"]), 0, 0) 
    accel_cols = [col for col in sample.columns if "_acceleration" in col]
    for col in accel_cols:
        sample[col] = scipy.signal.savgol_filter(sample[col], 20, 4)
    return (sample)

######### 2D ACCELERATION #############
def derive_accel2D_smooth(speedcols, sample):
    for col in speedcols:
        sample[col + '_acceleration2D'] = np.insert(np.diff(sample[col + "_speed2D"]), 0, 0) 
    accel_cols = [col for col in sample.columns if "_acceleration2D" in col]
    for col in accel_cols:
        sample[col] = scipy.signal.savgol_filter(sample[col], 20, 4)
    return (sample)

######### 3D JERK #############
def derive_jerk_smooth(speedcols, sample):
    # calcuates the jerk for each vertical column
    for col in speedcols:
        sample[col + '_jerk'] = np.insert(np.diff(sample[col + "_acceleration"]), 0, 0)
    # gets a list of all jerk columns
    jerk_cols = [col for col in sample.columns if "_jerk" in col]
    # smooths all jerk columns
    for col in jerk_cols:
        sample[col] = scipy.signal.savgol_filter(sample[col], 20, 4)
    return (sample)

######### 2D JERK #############
def derive_jerk2D_smooth(speedcols, sample):
    # calcuates the jerk for each vertical column
    for col in speedcols:
        sample[col + '_jerk2D'] = np.insert(np.diff(sample[col + "_acceleration2D"]), 0, 0)
    # gets a list of all jerk columns
    jerk_cols = [col for col in sample.columns if "_jerk2D" in col]
    # smooths all jerk columns
    for col in jerk_cols:
        sample[col] = scipy.signal.savgol_filter(sample[col], 20, 4)
    return (sample)


In [ ]:
# what is our sr
sr = 1/np.mean(np.diff(sample['time'])) # average time interval (difference) between consecutive frames
print(sr)

In what unit is this sampling rate?

In [ ]:
print(sr*1000)

And now?

Let's get all the 3D derivates on our sample file

In [ ]:
# compute derivatives

# we want to differentiate all columns except time
cols = sample.columns
cols = cols.drop(['time'])

# 3D speed
speedcols, sample = derive_speed_smooth(cols, sample, sr)

# 3D acceleration
sample = derive_accel_smooth(speedcols, sample)

# 3D jerk
sample = derive_jerk_smooth(speedcols, sample)

Now let's assume that you decide to proceed differently. You load in your sample, flip and interpolate it, but you decide to smooth the data only after you get the derivates - so that you can smooth all columns in one go.

Let's try it

In [ ]:
# get the same sample
sample_test = pd.read_csv(mtfiles[0])

# drop visibility cols
sample_test = sample_test.drop(sample_test.columns[sample_test.columns.str.contains('visibility',case = False)],axis = 1)

# flip the data
sample_test = flip_data(sample_test)

# interpolate the data
sample_test = interpolate(sample_test)

# now derive
cols = sample_test.columns
cols = cols.drop(['time'])

# 3D speed
speedcols, sample_test = derive_speed_smooth(cols, sample_test, sr)

# 3D acceleration
sample_test = derive_accel_smooth(speedcols, sample_test)

# 3D jerk
sample_test = derive_jerk_smooth(speedcols, sample_test)

Now let's inspect whether the order matters

This is speed of first smoothed, then derivated data

In [ ]:
sample['RIGHT_WRIST_speed'].plot()
plt.legend()
plt.show()

This is speed of first derivated, then smoothed data

In [ ]:
sample_test['RIGHT_WRIST_speed'].plot()
plt.legend()
plt.show()

Now let's look at the acceleration of the first smoothed, then derivated data

In [ ]:
sample['RIGHT_WRIST_acceleration'].plot()
plt.legend()
plt.show()

And now the acceleration of the first derivated, then smoothed data

In [ ]:
sample_test['RIGHT_WRIST_acceleration'].plot()
plt.legend()
plt.show()

What do you see?

Yes, the order matters. Differentiating multiplies the noise present in a signal, so before **every** differentiation, we want to make sure our data are *relatively* clean. 

So now we performed all steps that are necessary to get nice, clean data ready for analysis - on a **single** file. But we have more of them ready.

Luckily, because we have all the steps in functions, we can just apply them to all files in a loop.

If we know look at the sample dataframe, is there something we might be missing for future use?

In [ ]:
sample.head(20)

# Final code to process every file in Output_TimeSeries

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import platform  # To detect the operating system

# Get the current working directory
curfolder = os.getcwd()

# Check the OS type and set folder paths accordingly
if user == "Windows":
    mtfolder = curfolder + '\\..\\..\\01_MotionCapture\\Mediapipe\\Output_TimeSeries\\'
    processedfolder = curfolder + '\\TS_processed\\'
elif user == "Mac":  # Darwin is macOS
    mtfolder = os.path.join(curfolder, '..', '..', '01_MotionCapture', 'Mediapipe', 'Output_TimeSeries')
    processedfolder = os.path.join(curfolder, 'TS_processed')
else:
    raise SystemError("Unsupported operating system")

# Get all the files in the folder
mtfiles = glob.glob(os.path.join(mtfolder, '*body_world.csv'))

# If the folder TS_processed does not exist, create it
if not os.path.exists(processedfolder):
    os.makedirs(processedfolder, exist_ok=True)

for file in mtfiles:
    fileID = os.path.basename(file).split('.')[0]  # More OS-friendly way to extract fileID
    print('working on:', fileID)

    # Load the file
    sample = pd.read_csv(file)

    # Drop visibility columns
    sample = sample.drop(sample.columns[sample.columns.str.contains('visibility', case=False)], axis=1)

    # Flip the data
    sample = flip_data(sample)

    # Interpolate the data
    sample = interpolate(sample)

    # Smooth the data
    sample = smooth_data(sample)

    # Derive speed, acceleration, jerk
    cols = sample.columns.drop(['time'])
    speedcols, sample = derive_speed_smooth(cols, sample, sr)
    sample = derive_accel_smooth(speedcols, sample)
    sample = derive_jerk_smooth(speedcols, sample)

    # Save it
    sample.to_csv(os.path.join(processedfolder, f'{fileID}.csv'), index=False)

